In [2]:
import pandas as pd
from get_krx_value import get_krx_mean

In [3]:
df_price = (
        pd.read_pickle("./df_price_price_only_f_20130101_t_20230919.pkl")
        .loc[lambda df : df["code"].str[5]== "0"]
        # .loc[lambda df : ~df["name"].str.contains("스펙")]
        # .loc[lambda df : ~df["name"].str.contains("스팩")]    
    )

df_price['date'] = df_price['date'].dt.strftime("%Y-%m-%d")

try:
    df_krx = pd.read_pickle("./df_krx.pkl")
except :
    df_krx = get_krx_mean()
    df_krx.to_pickle("./df_krx.pkl")

df_price = df_price.merge(
    df_krx,
    on='date'
)

Index(['date', 'kospi', 'kosdaq'], dtype='object')


In [8]:
df_price.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'change', 'code',
       'name', 'marcap', 'kospi', 'kosdaq'],
      dtype='object')

In [4]:
def get_rolling_sma(cols, rolling_window):
        return (
            df_price.set_index("date")
            .groupby('code')[cols]
            .rolling(rolling_window)
            .mean()
            .unstack()
            .T
        )

In [9]:
change_pivot = get_rolling_sma('change', 1)

In [10]:
change_pivot.tail()

code,000020,000040,000050,000070,000080,000100,000120,000140,000150,000180,...,900340,950110,950130,950140,950160,950170,950190,950200,950210,950220
date,,,,,,,,,,,,,,,,,,,,,
2023-09-13,-0.008780,0.016845,0.011390,0.001429,0.015408,-0.056194,-0.005019,0.002334,-0.070866,0.000000,...,-0.022267,-0.006000,0.084211,-0.048061,-0.011957,-0.016049,-0.006098,-0.006803,-0.022500,-0.001070
2023-09-14,0.018701,-0.027108,0.002252,-0.001427,0.002529,0.027064,0.016393,-0.003492,0.018490,0.004938,...,-0.007246,0.016097,0.025596,0.052259,-0.003300,0.011292,-0.004382,-0.006849,0.025575,-0.012848
2023-09-15,-0.003865,0.077399,-0.006742,0.015714,0.016650,-0.005270,-0.009926,0.008178,0.012859,0.002457,...,0.005214,-0.011881,-0.018933,0.005892,-0.011038,0.017370,-0.003521,0.010345,-0.020781,-0.002711
2023-09-18,-0.012609,0.056034,-0.001131,-0.004219,0.004963,0.011921,0.072682,0.002317,-0.020911,-0.014706,...,-0.014523,-0.002004,-0.008772,0.056067,0.002232,-0.023171,-0.010601,0.027304,-0.007640,-0.013051
2023-09-19,-0.004912,0.040816,0.016988,-0.002825,-0.004938,0.022251,-0.014019,-0.001156,-0.095347,-0.009950,...,0.000000,0.016064,0.053982,-0.007924,-0.001114,0.023720,-0.002679,-0.028239,-0.012831,-0.008815


In [ ]:
change_pivot.rolling()

In [11]:
df_sample = df_price.loc[lambda df : df.code =="000020"]

In [12]:
df_sample.tail()

,date,open,high,low,close,volume,change,code,name,marcap,kospi,kosdaq
1499938,2023-09-13,10270,10390,10150,10160,74634,-0.008780,000020,동화약품,282945791100,2534.70,882.72
1499939,2023-09-14,10200,10360,10100,10350,83764,0.018701,000020,동화약품,282945791100,2572.89,899.47
1499940,2023-09-15,10350,10350,10190,10310,61011,-0.003865,000020,동화약품,282945791100,2601.28,899.03
1499941,2023-09-18,10200,10310,10150,10180,45588,-0.012609,000020,동화약품,282945791100,2574.72,891.29
1499942,2023-09-19,10120,10260,10110,10130,60080,-0.004912,000020,동화약품,282945791100,2559.21,883.89


In [15]:
delta = df_sample['close'].diff()
delta.head(10)

1497303      NaN
1497304    -90.0
1497305     20.0
1497306      0.0
1497307     90.0
1497308     90.0
1497309      0.0
1497310    130.0
1497311    -50.0
1497312    -30.0
Name: close, dtype: float64

In [16]:
delta.clip(lower=0)

1497303      NaN
1497304      0.0
1497305     20.0
1497306      0.0
1497307     90.0
           ...  
1499938      0.0
1499939    190.0
1499940      0.0
1499941      0.0
1499942      0.0
Name: close, Length: 2640, dtype: float64

In [17]:
delta.clip(upper=0)

1497303      NaN
1497304    -90.0
1497305      0.0
1497306      0.0
1497307      0.0
           ...  
1499938    -90.0
1499939      0.0
1499940    -40.0
1499941   -130.0
1499942    -50.0
Name: close, Length: 2640, dtype: float64

In [ ]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a given pandas Series and window period.

    Parameters:
    - data: pandas Series, the price data to calculate RSI for.
    - window: int, the period to use for calculating RSI, typically 14.

    Returns:
    - rsi: pandas Series, the RSI values.
    """
    # Calculate daily returns
    delta = data.diff()

    # Make two series: one for gains and one for losses
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # Calculate the average gain and loss
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Calculate the RS
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [ ]:
def calc_rsi_pivot(df_pivot, rsi_window, roll_window):
    delta_pivot = df_pivot.diff()
    gain_pivot = delta_pivot.clip(lower=0)
    loss_pivot = -delta_pivot.clip(upper=0)

    avg_gain_pivot = gain_pivot.rolling(window=rsi_window).mean()
    avg_loss_pivot = loss_pivot.rolling(window=rsi_window).mean()

    rs_pivot = avg_gain_pivot / avg_loss_pivot
    rsi_pivot = 100 - (100 / (1+rs_pivot))

    return rsi_pivot.rolling(roll_window).mean()